<a href="https://colab.research.google.com/github/tranamx3/NLP-Model-Analysis/blob/main/NLP_Model_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import glob
import os
import numpy as np
from sklearn.model_selection import train_test_split
import librosa
import soundfile
import pickle

# Define the dictionary of emotions
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# Specify the emotions that we are interested in
observed_emotions = ['calm', 'happy', 'surprised', 'sad']

# Function to extract features from each audio file
def extract_feature(file_name, mfcc, chroma, mel, max_pad_len=174):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        result = np.array([])

        if chroma:
            stft = np.abs(librosa.stft(X))
            chroma_feature = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            chroma_feature = np.pad(chroma_feature, (0, max_pad_len - len(chroma_feature)), 'constant')
            result = np.hstack((result, chroma_feature)) if result.size else chroma_feature

        if mfcc:
            mfcc_feature = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            mfcc_feature = np.pad(mfcc_feature, (0, max_pad_len - len(mfcc_feature)), 'constant')
            result = np.hstack((result, mfcc_feature)) if result.size else mfcc_feature

        if mel:
            mel_feature = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            mel_feature = np.pad(mel_feature, (0, max_pad_len - len(mel_feature)), 'constant')
            result = np.hstack((result, mel_feature)) if result.size else mel_feature

        return result

# Function to load the data, extract features for each sound file, and split the dataset
def load_data(test_size=0.2, max_pad_len=174):
    x, y = [], []
    path = '/content/drive/Shared drives/Social Domain Audios/Actors/*.wav'
    files = glob.glob(path)
    print(f"Found {len(files)} files")
    for file in files:
        file_name = os.path.basename(file)
        emotion_code = file_name.split("-")[2]
        emotion = emotions.get(emotion_code, None)
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True, max_pad_len=max_pad_len)
        if feature.size == 0:
            print(f"Skipping file {file_name} due to invalid feature extraction.")
            continue
        if len(feature) != 3 * max_pad_len:
            print(f"Feature length mismatch in file {file_name}: Expected {3 * max_pad_len}, got {len(feature)}")
            continue
        x.append(feature)
        y.append(emotion)

    if not x:
        raise ValueError("No valid features were extracted. Check the audio files and feature extraction.")

    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

# Example of how to use the load_data function
x_train, x_test, y_train, y_test = load_data(test_size=0.25)
print("Data loaded and split successfully")

Mounted at /content/drive
Found 1059 files


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=2
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Feature length mismatch in file 03-01-08-01-02-02-01.wav: Expected 522, got 174
Data loaded and split successfully


In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(359, 120)


In [ ]:
#Get the number of features extracted # check=======
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 522


In [ ]:
pip install scikit-learn

In [ ]:
from sklearn.neural_network import MLPClassifier
# Now you can use MLPClassifier
classifier = MLPClassifier()

from sklearn.metrics import accuracy_score


In [ ]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
# Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
#Calculate the accuracy of our model
accuracy= accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 84.17%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
f1_score(y_test, y_pred, average=None)

array([0.72727273, 0.76470588, 0.72340426])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted' :y_pred})
df.head(15)

,Actual,Predicted
0,sad,sad
1,happy,happy
2,sad,sad
3,surprised,surprised
4,happy,happy
5,surprised,surprised
6,sad,sad
7,sad,sad
8,sad,sad
9,happy,surprised
